In [1]:
## https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c30c1dcf2bc20119bcda7e734ce0eb42/quickstart_tutorial.ipynb#scrollTo=9eMtlafIw09I

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor








In [5]:
training_data  = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader  = DataLoader(test_data,batch_size=batch_size)


for X,y in test_dataloader : 
    print(f'Shape of X X[N, C, H ,W]: {X.shape}')
    print(f'Shape of y: {y.shape}')
    break



Shape of X X[N, C, H ,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [11]:
#torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using {device} device')

Using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)            
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)





In [22]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model.forward(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0 :
            loss , current = loss.item(), batch * len(X)
            print(f'loss {loss:>7f} [{current:>5d} / {size:>5d}]')
            

In [24]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss , correct = 0 , 0 
    with torch.no_grad():
        for X , y in dataloader:
            X , y = X.to(device) , y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size
    print(f'Test Error :\n Accuracy : {(100 * correct):>0.1f}% , Avg loss {test_loss:>8f} \n')


In [25]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1}\n ------------------------------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done")


Epoch 1
 ------------------------------------
loss 2.159784 [    0 / 60000]
loss 2.142291 [ 6400 / 60000]
loss 2.087816 [12800 / 60000]
loss 2.107653 [19200 / 60000]
loss 2.044547 [25600 / 60000]
loss 1.990269 [32000 / 60000]
loss 2.015966 [38400 / 60000]
loss 1.935310 [44800 / 60000]
loss 1.937897 [51200 / 60000]
loss 1.862387 [57600 / 60000]
Test Error :
 Accuracy : 54.8% , Avg loss 1.868545 

Epoch 2
 ------------------------------------
loss 1.903074 [    0 / 60000]
loss 1.866718 [ 6400 / 60000]
loss 1.752273 [12800 / 60000]
loss 1.799428 [19200 / 60000]
loss 1.676476 [25600 / 60000]
loss 1.636903 [32000 / 60000]
loss 1.660043 [38400 / 60000]
loss 1.566234 [44800 / 60000]
loss 1.587080 [51200 / 60000]
loss 1.481062 [57600 / 60000]
Test Error :
 Accuracy : 61.3% , Avg loss 1.509987 

Epoch 3
 ------------------------------------
loss 1.578828 [    0 / 60000]
loss 1.541597 [ 6400 / 60000]
loss 1.394063 [12800 / 60000]
loss 1.466693 [19200 / 60000]
loss 1.346043 [25600 / 60000]
loss 1